In [120]:
!pip install tqdm

  Using cached tqdm-4.64.1-py2.py3-none-any.whl (78 kB)


In [121]:
import os 
from tqdm.notebook import tqdm

In [16]:
ls

README.md                    output/
convertPDF2txt.ipynb         race-ethnicity-extractor.py
data/


In [14]:
!mkdir output

In [15]:
folder_dir = 'data/haotest/new_2000'
output_dir = 'output'

In [9]:
rm $folder_dir/*.nxml

In [20]:
ls $folder_dir

10642947.pdf  10714099.pdf  10881651.pdf  10977121.pdf  10992732.pdf
10642948.pdf  10714100.pdf  10881652.pdf  10983697.pdf  10992733.pdf
10642949.pdf  10714101.pdf  10881653.pdf  10983698.pdf  10998387.pdf
10642950.pdf  10714102.pdf  10895978.pdf  10983699.pdf  10998389.pdf
10642951.pdf  10719818.pdf  10895979.pdf  10983700.pdf  11017899.pdf
10642952.pdf  10719819.pdf  10895980.pdf  10983701.pdf  11049824.pdf
10646873.pdf  10719820.pdf  10895981.pdf  10983702.pdf  11051188.pdf
10646879.pdf  10719821.pdf  10895982.pdf  10983703.pdf  11051190.pdf
10653569.pdf  10719822.pdf  10895983.pdf  10983704.pdf  11051191.pdf
10653575.pdf  10719823.pdf  10895984.pdf  10987399.pdf  11056753.pdf
10653577.pdf  10735512.pdf  10895985.pdf  10987400.pdf  11056757.pdf
10653578.pdf  10753098.pdf  10903623.pdf  10987401.pdf  11068991.pdf
10661635.pdf  10756156.pdf  10905971.pdf  10987402.pdf  11068998.pdf
10661636.pdf  10772524.pdf  10905972.pdf  10987403.pdf  11068999.pdf
10661637.pdf  10803698.pdf  109059

In [12]:
!pip install pdfplumber
import pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 52.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.6/143.6 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 60.6 MB/s eta 0:00:00a 0:00:01


In [145]:
def pdfToTxt(pdf_file_dir, output_dir): # gets path of pdf file and the new path we want to put the text file
    no_text_converted = True 
    pdf_path=pdf_file_dir
    print(pdf_path)
    pdf_file_name = os.path.basename(pdf_file_dir)
    txt_file_name = pdf_file_name.replace('.pdf', '.txt')
    txt_path=os.path.join(output_dir, txt_file_name)
    pdf = pdfplumber.open(pdf_path) # opens the pdf and the pages
    pages = pdf.pages
#     print(len(pages))
    text_all = []
    race_info = []
    for page in pages: # goes through each page
        text = page.extract_text()
        if len(text)>0:
            text_all.append(text)
    if len(text_all)>0:
        no_text_converted = False

        text_all = [x.replace('\n', ' ').replace('\r', ' ') for x in text_all]
        text_all = " ".join(text_all)
    #     print(text_all)
        txt_file=open(txt_path, mode='w',encoding='utf-8') # creates the text file, writes the text into the text file, and closes it
        txt_file.write(text_all)
        race_info = get_races(text_all)
        if len(race_info)>0:
            print(race_info)
    else:
        print('no text')
    pdf.close()
    return text_all, race_info, no_text_converted
    
text_all, race_info, no_text_converted = pdfToTxt(folder_dir +'/11097375.pdf', output_dir)
print(text_all)
print(race_info)
print(no_text_converted)



../case-report-race-data/haotest/new_2000/11097375.pdf
Virchows Arch (2000) 437:459–464 ©Springer-Verlag 2000 CASE REPORT Konnie M. Hebeda · Marius A. MacKenzie J.Han.J.M. van Krieken A case of anaplastic lymphoma kinase-positive anaplastic large cell lymphoma presenting with spontaneous splenic rupture: an extremely unusual presentation Received: 24 February 2000 / Accepted: 27 April 2000 Abstract In a 22-year-old male with a 10-day history to date [11]. Therefore, splenic rupture as one of the pre- of fever, painful swelling in the left groin, and abdomi- senting symptoms in ALK-positive ALCL (small cell nal complaints, emergency surgery was performed be- variant) is exceptional. cause of spontaneous splenic rupture. At histology, a Recently,  a  subgroup  of  ALCL  was  defined  using cellular infiltrate of intermediate-sized atypical lympho- ALK1 staining. The ALK1 monoclonal antibody recog- cytes was seen in the splenic white pulp, staining for T- nizes an epitope of a fusion prot

In [148]:
files_no_text = []
files_no_race = []
files_has_race = []

In [149]:
df_list = []

In [ ]:
for year in range(2000, 2023):

    folder_dir = '../case-report-race-data/haotest/new_'+ str(year)
    output_dir = '../case-report-race-data/output/new_'+ str(year)
    os.makedirs(output_dir, exist_ok=True)


    pdf_files = []
    for root, dirs, files in os.walk(folder_dir):
        for afile in files:
            if afile.endswith(".pdf"):
                 pdf_files.append(os.path.join(root, afile))

    print(pdf_files[:10])
    
    print(len(pdf_files))
    for apdf in tqdm(pdf_files):
        text_all, race_info, no_text_converted = pdfToTxt(apdf, output_dir)
        if no_text_converted:
            files_no_text.append(apdf)
        else:
            if len(race_info)>0:
                files_has_race.append(apdf)
                print(race_info)
            else:
                files_no_race.append(apdf)
        pmid = os.path.basename(apdf).replace('.pdf', '')
        adflist=[pmid, year, race_info, no_text_converted]
        df_list.append(adflist)

    print(len(files_no_text))

    print(len(files_no_race))

    print(len(files_has_race))

['../case-report-race-data/haotest/new_2000/11097375.pdf', '../case-report-race-data/haotest/new_2000/10881652.pdf', '../case-report-race-data/haotest/new_2000/10881653.pdf', '../case-report-race-data/haotest/new_2000/10881647.pdf', '../case-report-race-data/haotest/new_2000/10735512.pdf', '../case-report-race-data/haotest/new_2000/10694069.pdf', '../case-report-race-data/haotest/new_2000/10694080.pdf', '../case-report-race-data/haotest/new_2000/10983697.pdf', '../case-report-race-data/haotest/new_2000/10719818.pdf', '../case-report-race-data/haotest/new_2000/10905974.pdf']
144


  0%|          | 0/144 [00:00<?, ?it/s]

../case-report-race-data/haotest/new_2000/11097375.pdf
../case-report-race-data/haotest/new_2000/10881652.pdf
../case-report-race-data/haotest/new_2000/10881653.pdf
../case-report-race-data/haotest/new_2000/10881647.pdf
../case-report-race-data/haotest/new_2000/10735512.pdf
['white girl']
['white girl']
../case-report-race-data/haotest/new_2000/10694069.pdf
../case-report-race-data/haotest/new_2000/10694080.pdf
['japanese  woman', 'japanese woman']
['japanese  woman', 'japanese woman']
../case-report-race-data/haotest/new_2000/10983697.pdf
no text
../case-report-race-data/haotest/new_2000/10719818.pdf
no text
../case-report-race-data/haotest/new_2000/10905974.pdf
../case-report-race-data/haotest/new_2000/10881651.pdf
../case-report-race-data/haotest/new_2000/10881650.pdf
../case-report-race-data/haotest/new_2000/10907780.pdf
../case-report-race-data/haotest/new_2000/10719819.pdf
no text
../case-report-race-data/haotest/new_2000/10646879.pdf
../case-report-race-data/haotest/new_2000/107

../case-report-race-data/haotest/new_2000/10905986.pdf
../case-report-race-data/haotest/new_2000/11068998.pdf
no text
../case-report-race-data/haotest/new_2000/10895985.pdf
no text
../case-report-race-data/haotest/new_2000/10714100.pdf
../case-report-race-data/haotest/new_2000/10803703.pdf
no text
../case-report-race-data/haotest/new_2000/10987403.pdf
../case-report-race-data/haotest/new_2000/10895978.pdf
no text
../case-report-race-data/haotest/new_2000/10983698.pdf
no text
../case-report-race-data/haotest/new_2000/10987401.pdf
../case-report-race-data/haotest/new_2000/10714102.pdf
../case-report-race-data/haotest/new_2000/11131347.pdf
../case-report-race-data/haotest/new_2000/10987400.pdf
../case-report-race-data/haotest/new_2000/10987399.pdf
../case-report-race-data/haotest/new_2000/10983699.pdf
no text
../case-report-race-data/haotest/new_2000/10895979.pdf
no text
43
81
20
['../case-report-race-data/haotest/new_2001/11196731.pdf', '../case-report-race-data/haotest/new_2001/11266308

  0%|          | 0/208 [00:00<?, ?it/s]

...

../case-report-race-data/haotest/new_2004/15225352.pdf
../case-report-race-data/haotest/new_2004/15050946.pdf
../case-report-race-data/haotest/new_2004/15363105.pdf
../case-report-race-data/haotest/new_2004/15182380.pdf
../case-report-race-data/haotest/new_2004/15566596.pdf
['caucasian man', 'caucasian man']
['caucasian man', 'caucasian man']
../case-report-race-data/haotest/new_2004/15137260.pdf
../case-report-race-data/haotest/new_2004/15150562.pdf
../case-report-race-data/haotest/new_2004/14725266.pdf
['chinese man']
['chinese man']
../case-report-race-data/haotest/new_2004/15116709.pdf
../case-report-race-data/haotest/new_2004/15153241.pdf
../case-report-race-data/haotest/new_2004/15385052.pdf
['caucasian female', 'caucasian woman']
['caucasian female', 'caucasian woman']
../case-report-race-data/haotest/new_2004/15217518.pdf
['japanese patient']
['japanese patient']
../case-report-race-data/haotest/new_2004/14966356.pdf
../case-report-race-data/haotest/new_2004/15366646.pdf
no tex

  0%|          | 0/501 [00:00<?, ?it/s]

../case-report-race-data/haotest/new_2005/15984719.pdf
../case-report-race-data/haotest/new_2005/16362650.pdf
../case-report-race-data/haotest/new_2005/16308476.pdf
../case-report-race-data/haotest/new_2005/16285034.pdf
../case-report-race-data/haotest/new_2005/16202173.pdf
../case-report-race-data/haotest/new_2005/16075155.pdf
../case-report-race-data/haotest/new_2005/16361826.pdf
../case-report-race-data/haotest/new_2005/15757574.pdf
../case-report-race-data/haotest/new_2005/16100469.pdf
../case-report-race-data/haotest/new_2005/16168058.pdf
../case-report-race-data/haotest/new_2005/15631632.pdf
['caucasian male', 'caucasian male']
['caucasian male', 'caucasian male']
../case-report-race-data/haotest/new_2005/16120008.pdf
../case-report-race-data/haotest/new_2005/15987402.pdf
../case-report-race-data/haotest/new_2005/16321153.pdf
../case-report-race-data/haotest/new_2005/15714667.pdf
../case-report-race-data/haotest/new_2005/16121882.pdf
../case-report-race-data/haotest/new_2005/1612

../case-report-race-data/haotest/new_2005/16355511.pdf
../case-report-race-data/haotest/new_2005/16219096.pdf
../case-report-race-data/haotest/new_2005/16010750.pdf
../case-report-race-data/haotest/new_2005/15705330.pdf
['japanese child', 'japanese girl']
['japanese child', 'japanese girl']
../case-report-race-data/haotest/new_2005/15651775.pdf
../case-report-race-data/haotest/new_2005/15906963.pdf
../case-report-race-data/haotest/new_2005/15632985.pdf
../case-report-race-data/haotest/new_2005/15882450.pdf
../case-report-race-data/haotest/new_2005/15655075.pdf
../case-report-race-data/haotest/new_2005/16235768.pdf
['caucasian male']
['caucasian male']
../case-report-race-data/haotest/new_2005/15906961.pdf
../case-report-race-data/haotest/new_2005/15791976.pdf
['caucasianfemale']
['caucasianfemale']
../case-report-race-data/haotest/new_2005/15988816.pdf
../case-report-race-data/haotest/new_2005/16042772.pdf
../case-report-race-data/haotest/new_2005/15651777.pdf
../case-report-race-data/

../case-report-race-data/haotest/new_2005/15716623.pdf
['korean child', 'korean child', 'korean child', 'korean girl', 'korean child', 'korean male', 'korean child']
['korean child', 'korean child', 'korean child', 'korean girl', 'korean child', 'korean male', 'korean child']
../case-report-race-data/haotest/new_2005/15744824.pdf
../case-report-race-data/haotest/new_2005/16159401.pdf
../case-report-race-data/haotest/new_2005/16355297.pdf
../case-report-race-data/haotest/new_2005/16270771.pdf
['japanese patient']
['japanese patient']
../case-report-race-data/haotest/new_2005/15763914.pdf
../case-report-race-data/haotest/new_2005/16135256.pdf
../case-report-race-data/haotest/new_2005/15663848.pdf
../case-report-race-data/haotest/new_2005/16229770.pdf
../case-report-race-data/haotest/new_2005/15962197.pdf
../case-report-race-data/haotest/new_2005/16134772.pdf
['asian  men', 'asian  patient', 'asian  men']
['asian  men', 'asian  patient', 'asian  men']
../case-report-race-data/haotest/new_

144


  0%|          | 0/144 [00:00<?, ?it/s]

../case-report-race-data/haotest/new_2000/11097375.pdf
6
[]
../case-report-race-data/haotest/new_2000/10881652.pdf
3
[]
../case-report-race-data/haotest/new_2000/10881653.pdf
2
[]
../case-report-race-data/haotest/new_2000/10881647.pdf
3
[]
../case-report-race-data/haotest/new_2000/10735512.pdf
7
['white girl']
['white girl']
../case-report-race-data/haotest/new_2000/10694069.pdf
8
[]
../case-report-race-data/haotest/new_2000/10694080.pdf
4
['japanese  woman', 'japanese woman']
['japanese  woman', 'japanese woman']
../case-report-race-data/haotest/new_2000/10983697.pdf
3
no text
../case-report-race-data/haotest/new_2000/10719818.pdf
4
no text
../case-report-race-data/haotest/new_2000/10905974.pdf
4
[]
../case-report-race-data/haotest/new_2000/10881651.pdf
2
[]
../case-report-race-data/haotest/new_2000/10881650.pdf
4
[]
../case-report-race-data/haotest/new_2000/10907780.pdf
6
[]
../case-report-race-data/haotest/new_2000/10719819.pdf
2
no text
../case-report-race-data/haotest/new_2000/106

no text
../case-report-race-data/haotest/new_2000/10714101.pdf
3
[]
../case-report-race-data/haotest/new_2000/10895984.pdf
5
no text
../case-report-race-data/haotest/new_2000/11068999.pdf
4
no text
../case-report-race-data/haotest/new_2000/10977120.pdf
7
[]
../case-report-race-data/haotest/new_2000/10983704.pdf
3
no text
../case-report-race-data/haotest/new_2000/10881649.pdf
2
[]
../case-report-race-data/haotest/new_2000/10881648.pdf
2
[]
../case-report-race-data/haotest/new_2000/10977121.pdf
7
[]
../case-report-race-data/haotest/new_2000/10905986.pdf
3
[]
../case-report-race-data/haotest/new_2000/11068998.pdf
4
no text
../case-report-race-data/haotest/new_2000/10895985.pdf
3
no text
../case-report-race-data/haotest/new_2000/10714100.pdf
3
[]
../case-report-race-data/haotest/new_2000/10803703.pdf
4
no text
../case-report-race-data/haotest/new_2000/10987403.pdf
4
[]
../case-report-race-data/haotest/new_2000/10895978.pdf
6
no text
../case-report-race-data/haotest/new_2000/10983698.pdf
3


In [132]:
print(files_no_text[:10])

print(files_no_race[:10])

print(files_has_race[:10])

['../case-report-race-data/haotest/new_2001/11306758.pdf', '../case-report-race-data/haotest/new_2001/11410703.pdf', '../case-report-race-data/haotest/new_2001/11289392.pdf', '../case-report-race-data/haotest/new_2001/11289393.pdf', '../case-report-race-data/haotest/new_2001/11289387.pdf', '../case-report-race-data/haotest/new_2001/11410702.pdf', '../case-report-race-data/haotest/new_2001/11410700.pdf', '../case-report-race-data/haotest/new_2001/11289391.pdf', '../case-report-race-data/haotest/new_2001/11289390.pdf', '../case-report-race-data/haotest/new_2001/11410701.pdf']
['../case-report-race-data/haotest/new_2001/11196731.pdf', '../case-report-race-data/haotest/new_2001/11641540.pdf', '../case-report-race-data/haotest/new_2001/11266334.pdf', '../case-report-race-data/haotest/new_2001/11752931.pdf', '../case-report-race-data/haotest/new_2001/11304002.pdf', '../case-report-race-data/haotest/new_2001/11564625.pdf', '../case-report-race-data/haotest/new_2001/11747736.pdf', '../case-rep

In [106]:
import re

# races_terms = 'pashtun|hispanic|tajik|hazara|uzbek|aimaks|turkmen|baloch|albanian|greeks|vlachs|gypsies|serbs|bulgarians|arabberber|european less than spanish|andorran|portuguese|french|ovimbundu|kimbundu|bakongo|mestico|european|black|british|portuguese|lebanese|syrian|white|mestizo|amerindian|armenian|russian|kurds|caucasian|asian|aborigina|croatians|slovenes|serbs|bosniaks|turks|german|azeri|dagestani|russian|armenian|armenians|nagornokarabakh region|black|white|bahraini|nonbahraini bengali|nonbengali muslims black|white|belorussian|russian|polish|ukrainian|fleming|walloon|or mestizo|creole|maya|garifuna|african|fon|adja|yoruba|bariba|europeans|bhote|ethnic nepalese|indigenous or migrant tribes quechua|mestizo|aymara|white bosniak|serb|croat|tswana |setswana|kalanga|basarwa|including kgalagadi and white| white|mulatto |white and black|black |includes japanese|arab|amerindian|unspecified malay|chinese|indigenous|bulgarian|turk|roma|including macedonian|armenian|tatar|circassian|mossi|gurunsi|senufo|lobi|bobo|mande|fulani hutu |bantu|tutsi |hamitic|twa|pygmy| khmer|vietnamese|chinese|cameroon highlanders|equatorial bantu|kirdi|fulani|northwest bantu|eastern nigritic|african|nonafrican less than british isles origin|french origin|european|indigenous indian and inuit|asian|african|arab|background creole |mulatto|african|european baya|banda|mandjia|sara|mboum|mbaka|yakoma|distinct groups north and center|mostly muslim|arabs|gorane |toubou|daza|kreda|zaghawa|kanembou|ouaddai|baguirmi|hadjerai|fulbe|kotoko|hausa|boulala|and maba south|sara |ngambaye|mbaye|goulaye|moundang|moussei|massa white and whiteamerindian|amerindian|han chinese|zhuang|uygur|hui|yi|tibetan|miao|manchu|mongol|buyi|korean|and nationalities mestizo|white|mulatto|black|blackamerindian|amerindian antalote|cafre|makoa|oimatsaha|sakalava with over african ethnic groups|bantu|mongo|luba|kongo|bantu|mangbetuazande|hamitic|kongo|sangha|mbochi|teke|europeans and white |including mestizo|black|amerindian|chinese|akan|voltaiques |gur|northern mandes|krous|southern mandes|lebanese|french| croat|serb|bosniak|hungarian|slovene|czech|roma|albanian|montenegrin|others mulatto|white|black|chinese greek|turkish |each concentrated almost exclusively in separate areas|czech|moravian|slovak|scandinavian|inuit|faroese|german|turkish|iranian|somali somali|afar|french|arab|ethiopian|and italian black|black and european|european|syrian|carib amerindian white|black|austronesian |malayopolynesian|papuan|small chinese minority mestizo |amerindian and white|amerindian|spanish and others|black egyptian|berber|nubian|bedouin|and beja|greek|armenian|european |primarily italian and french| mestizo|white|amerindian bioko |primarily bubi|some fernandinos|río muni |primarily fang|less than|europeans|mostly spanish ethnic tigrinya|tigre and kunama|afar|saho|estonian|russian|ukrainian|belorussian|finn|oromo|amhara and tigrean|sidamo|shankella|somali|afar|gurage|fijian|indian|european|pacific islanders|overseas chinese|and finn|swede|sami |lapp|roma|estonian celtic and latin with teutonic|slavic|north african|southeast asian|and basque minorities bantu tribes|fang|punu|nzeiby|mbede |obamba|bateke|africans and europeans|mandinka|fula|wolof|jola|serahuli|nonafrican georgian|azeri|armenian|russian|german|turkish|italian|greek|polish|black african|akan|moshidagomba|ewe|ga|gurma|yoruba |european and greek|black and european|european and east indian arawak|carib amerindian mestizo|ladino|amerindianspanish ancestry|and european|kiche|kaqchikel|mam|qeqchi|mayan|indigenous nonmayan|peuhl|malinke|susu|smaller tribes african |balanta|fula|manjaca|mandinga|black|amerindian |white|chinese|mulatto and white mestizo|amerindian|black|white hungarian|roma|or unknown homogeneous mixture of norse|celtic descendants|indoaryan|dravidian|mongoloid and javanese|sundanese|madurese|coastal malays|persian|azerbaijani|gilaki and mazandarani|kurd|arab|lur|baloch|turkmen|arab|kurdish|turkoman|assyrian|or celtic|english jewish |europe|americas|oceaniaborn|israelborn|africaborn|asiaborn |nonjewish |mostly arab|italian |includes small clusters of german|french|and sloveneitalians in the north and albanian and greekitalians in the south| black|east indian|white|chinese|japanese |korean|chinese|brazillian|filipino|arab|circassian|armenian kazak |qazaq|russian|ukrainian|uzbek|german|tatar|uygur|kikuyu |luhya |luo |kalenjin |kamba |kisii |meru |african |asian|european|kuwaiti|arab|south asian|iranian|kyrgyz|uzbek|russian|dungan|ukrainian|uygur|lao loum|lao theung|lao soung|hmong|meo|yao|mien|ethnic vietnamese|chinese latvian|russian|belorussian|ukrainian|polish|lithuanian|arab|armenian|sotho|europeans|asians|and indigenous african tribes |including kpelle|bassa|gio|kru|grebo|mano|krahn|gola|gbandi|loma|kissi|vai|bella|mandingo|and mende|americoliberians |descendants of former us slaves|congo people|descendants of former caribbean slaves| berber and arab|greeks|maltese|italians|egyptians|pakistanis|turks|indians|tunisians alemannic |italian|turkish|and lithuanian|polish|russian|or unspecified celtic base |with french and german blend|portuguese|italian|slavs |from montenegro|albania|and kosovo|and european |guest and worker residents| macedonian|albanian|turkish|roma |gypsy|serb|malayoindonesian |merina and related betsileo|côtiers |african|malayoindonesian|and arab ancestry|betsimisaraka|tsimihety|antaisaka|sakalava|french|indian|creole|comoran chewa|nyanja|tumbuko|yao|lomwe|sena|tonga|ngoni|ngonde|asian|european malay|chinese|indigenous|indian|south indians|sinhalese|arabs mande |bambara|malinke|sarakole|peul|voltaic|tuareg and moor|songhai|maltese |descendants of ancient carthaginians and phoenicians| micronesian maur|black|maur|black indomauritian|creole|sinomauritian|francomauritian mestizo |amerindianspanish|amerindian or predominantly amerindian|white|moldavian|romanian|ukrainian|russian|gagauz|bulgarian|french|monegasque|italian|mongol |khalkha|turkic|montenegrin|serbian|bosniak|albanian|muslims|croats|roma| arabberber|jewish|shangaan|chokwe|manyika|sena|makua|europeans|euroafricans|indians burman|shan|karen|rakhine|chinese|mon|indian|black|white|ovambo tribe|kavangos tribe|herero|damara|nama|caprivian|bushmen|baster|tswana nauruan|pacific islander|chinese|european brahmanhill|chetri|magar|tharu|tamang|newar|muslim|kami|yadav|mainly turks|moroccans|antilleans|surinamese|and indonesians|european|maori|pacific islander|asian|unspecified mestizo|white|black|amerindian hausa|djerma|fula|tuareg|beri beri |kanouri|arab|toubou|and gourmantche|yoruba|ibo|ijaw|kanuri|ibibio|tiv norwegian|sami|arab|baluchi|south asian |indian|pakistani|sri lankan|bangladeshi|african punjabi|sindhi|pashtun |pathan|baloch|muhajir|palauan |micronesian with malayan and melanesian admixtures|filipino|chinese| asian|white|carolinian|micronesian|or unspecified west bank|palestinian arab|jewish |gaza strip|palestinian arab and other|jewish mestizo|amerindian|west indian|white|indian melanesian|papuan|negrito|micronesian|polynesian mestizo amerindian|mestizo|white|black|japanese|chinese|tagalog|cebuano|ilocano|bisaya|binisaya|hiligaynon ilonggo|bikol|waray|polish|german|belorussian ukrainian|pakistani|indian|iranian|romanian|hungarian|roma |gyspy|ukrainian|german|russian|turkish'       
# race_terms = 'black|white|asian|hispanic|chinese|japanese|korean|french|indian|multiracial|latino|latina|latinx|african american|african|pacific islander|pacific|south american|alaska|alaskan native|indigenous|european|russian|caucasian|asian|british|portuguese|lebanese|syrian|german|polish|belorussian ukrainian|pakistani|indian|iranian|romanian|hungarian'
races_terms = 'latino|latina|latinx|pashtun|hispanic|tajik|hazara|uzbek|aimaks|turkmen|baloch|albanian|greeks|vlachs|gypsies|serbs|bulgarians|arabberber|andorran|portuguese|french|ovimbundu|kimbundu|bakongo|mestico|european|black|british|portuguese|lebanese|syrian|white|mestizo|amerindian|armenian|russian|kurds|caucasian|asian|aboriginal|croatians|slovenes|serbs|bosniaks|turks|german|azeri|dagestani|russian|armenian|armenians|nagornokarabakh region|black|white|bahraini|nonbahraini bengali|nonbengali muslims black|white|belorussian|russian|polish|ukrainian|fleming|walloon|creole|chinese|indigenous|bulgarian|turk|romamacedonian|armenian|tatar|circassian|fulani hutu|bantu|tutsi|hamitic|pygmy| khmer|vietnamese|chinese|cameroon highlanders|equatorial bantu|kirdi|fulani|northwest bantu|eastern nigritic|african|indigenous indian and inuit|asian|african|arab |mulatto|african|european baya|banda|mandjia|mboum|mbaka|yakoma|arabs|gorane|toubou|kreda|zaghawa|kanembou|ouaddai|baguirmi|hadjerai|fulbe|kotoko|hausa|boulala|sara |ngambaye|mbaye|goulaye|moundang|moussei|massa white and whiteamerindian|amerindian|han chinese|zhuang|uygur|tibetan|manchu|mongol|buyi|korean|white|mulatto|black|blackamerindian|cafre|makoa|oimatsaha|bantu|mongo|luba|kongo|bantu|mangbetuazande|hamitic|kongo|sangha|mbochi|europeans and white |including mestizo|black|amerindian|chinese|voltaiques|northern mandes|krous|southern mandes|lebanese|french|croat|serb|bosniak|hungarian|slovene|czech|romanian|albanian|montenegrin|white|black|chinese greek|turkish |czech|moravian|slovak|scandinavian|inuit|faroese|german|turkish|iranian|somali somali|afar |french|arab |ethiopian|italian black|black and european|european|syrian|carib amerindian white|black|austronesian |malayopolynesian|papuan|small chinese minority mestizo |amerindian and white|amerindians|black egyptian|berber|nubian|bedouin|beja|greek|armenian|european |primarily italian and french| mestizo|white|amerindian bioko|europeans|tigre and kunama|estonian|russian|ukrainian|belorussian|finn |oromo|amhara and tigrean|sidamo|shankella|somali|afar |gurage|fijian|indian|european|pacific islanders|overseas chinese|and finn|swede|sami |lapp |romanian|estonian celtic and latin with teutonic|slavic|north african|southeast asian|nzeiby|mbede|obamba|bateke|africans and europeans|mandinka|fula|wolof|jola|serahuli|nonafrican georgian|azeri|armenian|russian|german|turkish|italian|greek|polish|black african|akan|moshidagomba|gurma|yoruba european and greek|black and european|european and east indian arawak|carib amerindian mestizo|ladino|amerindianspanish ancestry|kiche|kaqchikel|qeqchi|mayan|indigenous nonmayan|peuhl|malinke|balanta|manjaca|mandinga|black|amerindian|white|chinese|mulatto and white mestizo|amerindian|black|white hungarian|indoaryan|dravidian|mongoloid and javanese|sundanese|madurese|coastal malays|persian|azerbaijani|kurd|arab |baloch|turkmen|arab |kurdish|turkoman|assyrian|celtic|europe|americas|oceania born|israel born|africa born|asia born|non jewish|italian|french|black|east indian|white|chinese|japanese|korean|chinese|brazillian|filipino|arab |circassian|armenian kazak |qazaq|russian|ukrainian|uzbek|german|tatar|uygur|african|asian|european|kuwaiti|arab |south asian|iranian|kyrgyz|uzbek|russian|dungan|ukrainian|uygur|lao loum|lao theung|lao soung|hmong|mien|ethnic vietnamese|chinese latvian|russian|belorussian|ukrainian|polish|lithuanian|arab |armenian|sotho|europeans|asians|greeks|maltese|italians|egyptians|pakistanis|turks|indians|tunisians alemannic|italian|turkish|lithuanian|polish|russian|portuguese|italian|slavs |montenegro|albania|kosovo|macedonian|albanian|turkish|romaniangypsy|serb|malayoindonesian |merina and related betsileo|african|malayoindonesian|arab ancestry|betsimisaraka|tsimihety|antaisaka|sakalava|french|indian|creole|comoran chewa|nyanja|tumbuko|yao|lomwe|sena|tonga|ngoni|ngonde|asian|european malay|chinese|indigenous|indian|south indians|sinhalese|arabs mande |bambara|malinke|sarakole|peul|voltaic|tuareg and moor|songhai|maltese |descendants of ancient carthaginians and phoenicians| micronesian maur|black|maur|black indomauritian|creole|sinomauritian|francomauritian mestizo |amerindianspanish|amerindian or predominantly amerindian|white|moldavian|romanian|ukrainian|russian|gagauz|bulgarian|french|monegasque|italian|mongol |khalkha|turkic|montenegrin|serbian|bosniak|albanian|muslims|croats|romanian| arabberber|jewish|shangaan|chokwe|manyika|sena|makua|europeans|euroafricans|indians burman|karen|rakhine|chinese|indian|black|white|ovambo tribe|kavangos tribe|herero|damara|nama|caprivian|bushmen|baster|tswana nauruan|pacific islander|chinese|european brahmanhill|chetri|tamang|newar|muslimi|yadavs|moroccans|antilleans|surinamese|european|maori|pacific islander|asian|unspecified mestizo|white|black|amerindian hausa|djerma|tuareg|beri beri |kanouri|arab |toubou|gourmantche|yoruba|south asian|indian|pakistani|sri lankan|bangladeshi|african punjabi|sindhi|pathan|baloch|muhajir|palauan|filipino|chinese|asian|white|carolinian|micronesian|jewish |gaza strip|jewish mestizo|amerindian|west indian|white|indian melanesian|papuan|negrito|micronesian|polynesian mestizo amerindian|mestizo|white|black|japanese|chinese|tagalog|cebuano|ilocano|bisaya|binisaya|hiligaynon ilonggo|bikol|waray|polish|german|belorussian ukrainian|pakistani|indian|iranian|romanian|hungarian|romanian|gyspy|ukrainian|german|russian|turkish'

patterns = f'({race_terms})([\w\s]'+'{,10})(girl|boy|patient|american|american|child|children|participants|girls|boys|patients|americans|male|males|female|females|participant|woman|women| man| men|subject|subjects|islander|islander|native|native|natives|mother|mothers|father|fathers|soldier)'
# patterns = f'(girl|boy|patient|american|american|child|children|participants|girls|boys|patients|americans|male|males|female|females|participant|woman|women| man| men|subject|subjects|islander|islander|native|native|natives|mother|mothers|father|fathers|soldier)'



matches = re.findall(patterns, text_all.lower())

for match in matches:
    print(''.join(match))


    
def get_races(text_all):
    matches = re.findall(patterns, text_all.lower())
    results = []
    for match in matches:
        results.append(''.join(match))
    return results

print(get_races(text_all))
    

[]


In [36]:
!pip install pdfminer.six

In [40]:

from pdfminer.high_level import extract_text

pdf_path=os.path.join(folder_dir, '10653577.pdf')

text = extract_text(pdf_path)
print(text)

Letters
Letters
Letters
Letters
Letters

Isolation of a Dengue Type 1 Virus from a
Soldier in West Africa (Côte d’Ivoire)

To  the  Editor:  In  Africa,  recent  reports  of
epidemic  or  endemic  dengue  activity  usually
come from the eastern part of the continent (1),
and  the  serotype  most  frequently  identified  is
dengue 2.

investigations  were  as 

We  report  the  isolation  of  a  dengue  1  virus
strain from the blood of a young soldier living in
Abidjan,  the  capital  of  Côte  d’Ivoire.  This
20-year-old man arrived from France on October
19, 1998. On December 28, 1998, he consulted a
physician  of  his  regiment  because  of  headache,
diarrhea,  and  fever  (40°C).  The  results  of  the
laboratory 
follows:
platelets  (193  109 /L),  leukocytes  (2.2109 /L);  no
malaria  was  found.  He  was  hospitalized  for
possible arboviral infection, and treatment with
paracetamol was prescribed. On December 29, a
blood sample was collected; serum and buffy coat
were frozen a

In [21]:
!ls output

10642947.txt


In [ ]:
import os
import tarfile
import pdfplumber
import re

def pdfToTxt(pdfFile,namewithoutending): # gets path of pdf file and the new path we want to put the text file
    path=("C:/Users/bzeli/OneDrive/Desktop/Sci_Re/CODING/download/2010/" + pdfFile)
    newpath="C:/Users/bzeli/OneDrive/Desktop/Sci_Re/CODING/download/txtfiles" + "/" + namewithoutending + ".txt"
    pdf=pdfplumber.open(path) # opens the pdf and the pages
    pages=pdf.pages 
    text_all=[]
    for page in pages: # goes through each page
        text=page.extract_text()
        text_all.append(text)
    text_all="".join(text_all)
    txt_file=open(newpath,mode='a',encoding='utf-8') # creates the text file, writes the text into the text file, and closes it
    txt_file.write(text_all)
    pdf.close()

def get_pdf_name(file_name):
  matches = re.match(r"^(PMC\d+)",file_name)
  if matches:
    return matches[0]

def compressedToPdf(tarfilename): # finds the tar file and opens it
    path=("C:/Users/bzeli/OneDrive/Desktop/Sci_Re/CODING/download/2010/" + tarfilename)
    tar = tarfile.open(path)
    pmid = ""

    for member in tar.getmembers():  # goes through the tar, finds the pdf file, gets the filename and pmid, and extracts it to the tarstopdf folder
        filename = member.name
        if filename.endswith('.pdf'):
            filename = get_pdf_name(filename)
            pmid = filename.split('/')[0] #gets pmid
            print(pmid)
            tar.extractall("C:/Users/bzeli/OneDrive/Desktop/Sci_Re/CODING/download/tarstopdf")
    return pmid # returns the pmid for when converting to pdf ()

files = os.listdir("C:/Users/bzeli/OneDrive/Desktop/Sci_Re/CODING/download/2010")

def tarToTxt(pdfFile, pmid):
    path=("C:/Users/bzeli/OneDrive/Desktop/Sci_Re/CODING/download/tarstopdf/" + pmid)
    # pdfFile = ""
    # for item in os.listdir(path):
    #     if os.path.isfile(os.path.join(path, f)) and f.endswith('.pdf'):
    #         pdfFile = item
    #finds the pdf file in the pdftotars folder and the new path for where the text file will be placed
    pdfFile = ""
    files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f)) and f.endswith('.pdf')]
    if files:
        pdfFile = files[0]
    else:
        return

    path_of_pdf = os.path.join(path, pdfFile)

    # new name of pdf file that we want to convert to txt
    pdfFile = pmid[3:] + ".pdf"
    new_path_of_pdf = os.path.join(path, pdfFile)

    # renames pdf file to be the pmid
    os.rename(path_of_pdf, new_path_of_pdf)

    newpath="C:/Users/bzeli/OneDrive/Desktop/Sci_Re/CODING/download/txtfiles" + "/" + pdfFile[:-4] + ".txt"
    pdf=pdfplumber.open(new_path_of_pdf)  # opens the pdf file from the tarstopdf folder and pages
    pages=pdf.pages 
    text_all=[]

    for page in pages:   # writes the text file with what was in the tar file
        text=page.extract_text()
        text_all.append(text)
    text_all="".join(text_all)
    txt_file=open(newpath,mode='a',encoding='utf-8')
    txt_file.write(text_all)
    pdf.close()

for x in range(0,len(files)):
    if files[x][-4:] == ".pdf":
        pdfToTxt(files[x],files[x][:-4])  # converts pdf to txt
    if files[x][-7:] == ".tar.gz":
        pmid = compressedToPdf(files[x])  # pmid from the function
        newName = files[x][:-7] + ".pdf"  # get the new name of the pdf
        tarToTxt(newName, pmid)  # creates the txt file from tar file